In [34]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [50]:
data = pd.read_csv('/Users/1hundredeyes/factors_cpi_new.csv')

In [52]:
data["month"] = pd.to_datetime(data["month"]) 
data.set_index("month", inplace = True) 
data = data.asfreq('MS') 

In [54]:
data = data.drop(data.loc['2023':].index)

In [56]:
data = np.log(data)

In [58]:
split_date = "2018-12-01"
data_train = data[:split_date] 
data_test = data[split_date:].iloc[1:]

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(data['xrt'],data['cpi'] , color='blue', s=20,  alpha=0.6, edgecolors='k')
slope, intercept, rvalue, pvalue, stderr = stats.linregress(data['xrt'],data['cpi'])
plt.plot(data['xrt'], slope * data['xrt'] + intercept, color='red')
plt.text(min(data['xrt']) + (max(data['xrt']) - min(data['xrt'])) * 0.05, max(data['cpi']) - (max(data['cpi']) - min(data['cpi'])) * 0.1, 
         f"$R^2$ = {rvalue ** 2:.2f}", fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Exchange Rate of Bath to USD')
plt.ylabel('Consumer Price Index')
plt.title('Consumer Price Index VS Exchange Rate of Bath to USD')
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(data['m2'],data['cpi'] , color='blue', s=20,  alpha=0.6, edgecolors='k')
slope, intercept, rvalue, pvalue, stderr = stats.linregress(data['m2'],data['cpi'])
plt.text(min(data['m2']) + (max(data['m2']) - min(data['m2'])) * 0.05, max(data['cpi']) - (max(data['cpi']) - min(data['cpi'])) * 0.1, 
         f"$R^2$ = {rvalue ** 2:.2f}", fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))
plt.plot(data['m2'], slope * data['m2'] + intercept, color='red')
plt.xlabel('Broad Money')
plt.ylabel('Consumer Price Index')
plt.title('Consumer Price Index VS Broad Money')
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(data['policy_rate'],data['cpi'] , color='blue', s=20,  alpha=0.6, edgecolors='k')
slope, intercept, rvalue, pvalue, stderr = stats.linregress(data['policy_rate'],data['cpi'])
plt.plot(data['policy_rate'], slope * data['policy_rate'] + intercept, color='red')
plt.text(min(data['policy_rate']) + (max(data['policy_rate']) - min(data['policy_rate'])) * 0.05, max(data['cpi']) - (max(data['cpi']) - min(data['cpi'])) * 0.1, 
         f"$R^2$ = {rvalue ** 2:.2f}", fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Policy Rate')
plt.ylabel('Consumer Price Index')
plt.title('Consumer Price Index VS Policy Rate')
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(data['export'],data['cpi'] , color='blue', s=20,  alpha=0.6, edgecolors='k')
slope, intercept, rvalue, pvalue, stderr = stats.linregress(data['export'],data['cpi'])
plt.plot(data['export'], slope * data['export'] + intercept, color='red')
plt.text(min(data['export']) + (max(data['export']) - min(data['export'])) * 0.05, max(data['cpi']) - (max(data['cpi']) - min(data['cpi'])) * 0.1, 
         f"$R^2$ = {rvalue ** 2:.2f}", fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Export')
plt.ylabel('Consumer Price Index')
plt.title('Consumer Price Index VS Export')
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(data['import'],data['cpi'] , color='blue', s=20,  alpha=0.6, edgecolors='k')
slope, intercept, rvalue, pvalue, stderr = stats.linregress(data['import'],data['cpi'])
plt.plot(data['import'], slope * data['import'] + intercept, color='red')
plt.text(min(data['import']) + (max(data['import']) - min(data['import'])) * 0.05, max(data['cpi']) - (max(data['cpi']) - min(data['cpi'])) * 0.1, 
         f"$R^2$ = {rvalue ** 2:.2f}", fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Import')
plt.ylabel('Consumer Price Index')
plt.title('Consumer Price Index VS Import')
plt.show()

In [ ]:
x_train, x_test = data_train[['xrt', 'm2', 'export', 'import']], data_test[['xrt', 'm2', 'export', 'import']]
y_train, y_test = data_train['cpi'], data_test['cpi']

In [62]:
data_train["m2_diff1"] = data_train["m2"].diff()
data_train.dropna(inplace = True)

/var/folders/j4/9cv4ksxd5p11m01b59qb0wyw0000gn/T/ipykernel_4929/3303962367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train["m2_diff1"] = data_train["m2"].diff()
/var/folders/j4/9cv4ksxd5p11m01b59qb0wyw0000gn/T/ipykernel_4929/3303962367.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.dropna(inplace = True)


In [ ]:
data_train["cpi_diff1"] = data_train["cpi"].diff()
data_train.dropna(inplace = True)

In [ ]:
def stepwise_selection(x_train, y_train, threshold_in=0.05, threshold_out=0.1):
    initial_features = list(x_train.columns)  # ใช้ x_train แทน X
    best_features = []

    while True:
        changed = False

        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features, dtype=float)
        for new_column in remaining_features:
            model = sm.OLS(y_train, sm.add_constant(x_train[best_features + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]

        if not new_pval.empty and new_pval.min() < threshold_in:
            best_feature = new_pval.idxmin()
            best_features.append(best_feature)
            changed = True

        # Backward Elimination: ลองลบตัวแปรที่มีค่า p-value สูงเกิน threshold_out
        model = sm.OLS(y_train, sm.add_constant(x_train[best_features])).fit()
        pvalues = model.pvalues.iloc[1:]  # ไม่รวม intercept
        worst_pval = pvalues.max()

        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            best_features.remove(worst_feature)
            changed = True

        if not changed:
            break

    return best_features

selected_features = stepwise_selection(x_train, y_train)
print("Selected Features:", selected_features)

In [84]:
model = SARIMAX(data_train['cpi'], exog = data_train[['m2']], order = (1, 1, 0), seasonal_order = (1, 1, 0, 12)
                , enforce_stationarity = False, enforce_invertibility = False)
model_fit = model.fit()
print(model_fit.summary())

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -3.33804D+00    |proj g|=  4.73127D+01

At iterate    5    f= -3.34692D+00    |proj g|=  4.25494D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      6     61      2     0     0   4.255D-01  -3.347D+00
  F =  -3.3469193808452906     

ABNORMAL_TERMINATION_IN_LNSRCH                              
                                     SARIMAX Results                                      
Dep. Variable:                           


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
